In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#drive.flush_and_unmount()

Mounted at /content/drive


In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path # Path module of pathlib is used to store path
import os
import cv2
from google.colab.patches import cv2_imshow

#Choix du dataset : 
data_set_path = "/content/drive/MyDrive/FS18-Luc-Fred-Nansa/dataset_updated"
#data_set_path = "/content/drive/MyDrive/FS18-Luc-Fred-Nansa/Data-for-test"

global_data = Path(data_set_path+'/global_set/') 
training_data = Path(data_set_path+'/training_set/') 
validation_data = Path(data_set_path+'/validation_set/') 

bad_dir = "bad"
notRGB_dir = "notRGB"

#Choose global_data / training_data / validation_data
data = global_data


In [15]:
# get the dataset labels 
labels_name = os.listdir(data)

# sort the training labesl 
labels_name.sort()
print(labels_name)


['drawings', 'engraving', 'iconography', 'painting', 'sculpture']


In [16]:
#--------------------------------------------------------------
# Suppression des fausses images du dataset (fichiers html ...)
#--------------------------------------------------------------
import shutil
def clean_dataset(labels, path):
    bad_path = path / Path("..") / Path(os.path.basename(path)+"_"+bad_dir)
    bad_cpt = 0

    # iterate the folder to get the image label name
    # loop over the sub folder 
    for name in labels_name:
        name_bad = name + "_" + bad_dir
        # join the training data path and each species training folder
        dir = os.path.join(path, name)

        for file in sorted(os.listdir(dir)):
            file_name = file
            file = dir + "/" + os.fsdecode(file)
       
            # read the image
            image = cv2.imread(file) 
        
            # Move the file if not an image
            if image is None:
                # Créé le dossier de classe s'il s'existe pas
                if not os.path.isdir(bad_path / name_bad):
                    os.makedirs(bad_path / name_bad)
                # Move the file
                shutil.move(file, str(bad_path / name_bad / file_name))
                print("Moved "+file+" to "+ str(bad_path / name_bad / file_name))
                bad_cpt+=1   

        print("Current processed folder: {}".format(name))

    print("Completed cleanning data set, "+str(bad_cpt)+" bad images found.")


In [17]:
!pip install Image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.9 MB 5.2 MB/s 
  Created wheel for Image: filename=image-1.5.33-py2.py3-none-any.whl size=19496 sha256=7e16a331a2c278e3e7b424bf0cf463c4adf5000b5d4855d4f2abe5844be6e6ca
  Stored in directory: /root/.cache/pip/wheels/56/88/e6/897194cfe8c08a8b9afd881d3bf53d102e13fa39607d721383
Successfully built Image


In [18]:
#--------------------------------------------------------------
# Detection et conversion RGBA des images avec composante de transparence
#--------------------------------------------------------------
import shutil
from PIL import Image

def convert_RGB_dataset(labels, path):
    notRGB_path = path / Path("..") / Path(os.path.basename(path)+"_"+notRGB_dir)
    notRGB_cpt = 0

    # itirate the folder to get the image label name
    # loop over the sub folder 
    for name in labels_name:
        name_notRGB_path = notRGB_path / Path(name + "_" + notRGB_dir)
        # join the training data path and each species training folder
        dir = os.path.join(path, name)

        for file in sorted(os.listdir(dir)):
            file_name = file
            file = dir + "/" + os.fsdecode(file)
       
            # read the image
            image = Image.open(file)
            # Convert the image if not RGB
            if image is not None:
                if image.mode is not 'RGB':
                    # Créé le dossier de classe s'il s'existe pas
                    if not os.path.isdir(name_notRGB_path):
                        os.makedirs(name_notRGB_path)
                        print("Create : "+str(name_notRGB_path))

                    print("Detected not RGB file : (" + image.mode + ') '+file)
                    notRGB_cpt+=1 
                    # Copy file
                    shutil.copy2(file, str(name_notRGB_path / file_name))
                    print("Copy "+file+" to "+ str(name_notRGB_path / file_name))      
                                  
                    image.convert("RGB").save(file)
                    print("Converted : "+file+" to "+ str(name_notRGB_path / file_name))

        print("Current processed folder: {}".format(name))

    print("Completed converting data set, "+str(notRGB_cpt)+" images converted.")

In [ ]:
#--------------------------------------------------------------
# Fusion training + validation => global datset 
#--------------------------------------------------------------
from distutils.dir_util import copy_tree

copy_tree ("/content/drive/MyDrive/FS18-Luc-Fred-Nansa/dataset_updated/training_set",
           "/content/drive/MyDrive/FS18-Luc-Fred-Nansa/dataset_updated/global_set")

copy_tree("/content/drive/MyDrive/FS18-Luc-Fred-Nansa/dataset_updated/validation_set",
                "/content/drive/MyDrive/FS18-Luc-Fred-Nansa/dataset_updated/global_set")

In [ ]:
# Clean the dataset
clean_dataset(labels_name, data)

# Convert non RGB to RGB
convert_RGB_dataset(labels_name, data)

Current processed folder: drawings
Current processed folder: engraving
Current processed folder: iconography
Current processed folder: painting
Current processed folder: sculpture
Completed converting data set, 0 images converted.
